In [ ]:
#Enhanced Dashboard Application
#Created by Andres Torres
# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from dash import Dash
import os
import dash_bootstrap_components as dbc

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "mypassword"

# Connect to database via CRUD Module
animals = AnimalShelter(username,password)
#animals = AnimalShelter(username, password)
# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(animals.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

df = pd.DataFrame.from_records(animals.read({}))
df.drop(columns=['_id'], inplace=True, errors='ignore')

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)

#########################
# Dashboard Layout / View
#########################
#app = Dash(__name__)
external_stylesheets = [
    'https://fonts.googleapis.com/css2?family=Cinzel&display=swap'
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())))
html.Center(html.Img(
    src='data:image/png;base64,{}'.format(encoded_image.decode()),
    alt='customer image',
    style={'width': '600px', 'height': 'auto', 'border': '4px solid #3B82F6', 'borderRadius': '12px'}
))

#app.layout = html.Div([
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(
            html.Center(html.B(html.H1("Grazioso Salvare Dashboard"))),style={'fontFamily': 'Cinzel, serif','fontSize': '40px'},
            width=10)
    ]),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center((html.B(html.H2('Welcome to'))),style={'fontFamily': 'Cinzel, serif','fontSize': '30px'}),
    html.Center((html.B(html.H2('Andres Torres Dashboard!'))),style={'fontFamily': 'Cinzel, serif','fontSize': '30px'}),
    html.Hr(),
    html.Hr(),
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image',style={'border': '4px solid #3B82F6', 'borderRadius': '12px', 'padding': '4px'})),
    html.Div(


    dbc.Card(
        dbc.CardBody([
            html.H5("Filter Type", className="card-title", style={'fontFamily': 'Cinzel, serif'}),
            dbc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                    {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                    {'label': 'Reset', 'value': 'reset'}
                ],
                value='reset',
                inline=True,
                className='mb-2'
            )
        ]),
        className="mb-4 border-primary shadow-sm p-2"
    ),
    ),
    html.Hr(),
    html.Hr(),

        # Tab System
    dbc.Tabs([
        dbc.Tab(label="Data Table", tab_id="tab-table"),
        dbc.Tab(label="Visualizations", tab_id="tab-graph"),
        dbc.Tab(label="Map", tab_id="tab-map")
    ], id="tabs", active_tab="tab-table"),
    
    # Where tab content will display
    html.Div(id="tab-content", className="p-4"),
    html.Hr(),
])

    

dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="multi",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,

                        ),
html.Br(),
html.Hr(),
html.Hr(),

#-----------------------------------------
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side

dbc.Row([
    dbc.Col(
        html.Div(id='graph-id', style={
            'border': '1px solid #ccc',
            'borderRadius': '8px',
            'padding': '10px',
            'backgroundColor': '#f9f9f9'
        }),
        width=6
    ),
    dbc.Col(
        html.Div(id='map-id', style={
            'border': '1px solid #ccc',
            'borderRadius': '8px',
            'padding': '10px',
            'backgroundColor': '#f9f9f9'
        }),
        width=6
    )
], className="mb-4 g-3")
#############################################
# Interaction Between Components / Controller
#############################################

#--------------------
#Table
#--------------------

@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value')],)
def update_dashboard(filter_type):
        # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))

        else:
            df = pd.DataFrame.from_records(animals.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
               
        return (data,columns)
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

#----------------------
# Graph
#----------------------

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or 'breed' not in dff.columns:
        # Return an empty figure or a friendly message
        return [dcc.Graph(figure={})]
    return [
        dcc.Graph(figure=px.histogram(dff, x='breed'))
    ]

#-----------------------
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

@app.callback(
    Output("tab-content", "children"),
    Input("tabs", "active_tab")
)
def render_tab_content(active_tab):
    if active_tab == "tab-table":
        return html.Div([
            dash_table.DataTable(
                id='datatable-id',
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict('records'),
                page_size=10
            )
        ])
    
    elif active_tab == "tab-graph":
        return html.Div([
            dcc.Graph(
                id='graph-id',
                figure=px.histogram(df, x='breed')
            )
        ])
    
    elif active_tab == "tab-map":
        return html.Div([
            dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[dl.TileLayer()]
            )
        ])

#----------------
# Map 
#----------------

def update_map(viewData, row_ids):
    # Provide default Austin, TX coordinates if no selection/data
    default_center = [30.75, -97.48]
    
    if not viewData or row_ids is None or len(row_ids) == 0:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=default_center, zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]
    
    dff = pd.DataFrame.from_dict(viewData)
    selected_row = row_ids[-1]
    # Ensure columns exist
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=default_center, zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]
    # Ensure values are not null/NaN
    lat = dff.loc[selected_row, 'location_lat']
    lon = dff.loc[selected_row, 'location_long']
    if pd.isnull(lat) or pd.isnull(lon):
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=default_center, zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]
    #------------------
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[lat, lon], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(str(dff.loc[selected_row, 'breed']) if 'breed' in dff.columns else ""),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(str(dff.loc[selected_row, 'name']) if 'name' in dff.columns else "")
                    ])
                ])
            ]
        )
    ]

if __name__ == '__main__':
    app.run(debug=True)

10002
Index(['name', 'species', 'adopted', '', 'age_upon_outcome', 'animal_id',
       'animal_type', 'breed', 'color', 'date_of_birth', 'datetime',
       'monthyear', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome',
       'location_lat', 'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
